In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

bands = pd.read_csv('data/ma_bands_data.csv', index_col=0).sort_values('date')

In [2]:
bands['theme'] = bands['theme'].fillna('')

In [3]:
bands

,name,url,genre,theme,label,country,location,status,date,years
84494,Olympic,https://www.metal-archives.com/bands/Olympic/3...,Pop Rock; Heavy Metal/Hard Rock (mid),,NaN,Czechia,Prague,Active,1962.0,1962-present
85413,Orient,https://www.metal-archives.com/bands/Orient/51156,Hard Rock (early); Heavy Metal (later),"['Sex', ' Evil']",Unsigned/independent,Czechia,Prague,Active,1964.0,"1964-1990,1998-present"
98592,Scorpions,https://www.metal-archives.com/bands/Scorpions...,Heavy Metal/Hard Rock,"['Life', ' Society', ' Love', ' Sex', ' Inner ...",NaN,Germany,"Hannover, Lower Saxony",Active,1964.0,1964-present
43631,Faithful Breath,https://www.metal-archives.com/bands/Faithful_...,Progressive Rock (early); Heavy Metal/Hard Roc...,"['Adventure', ' Fantasy', ' Women']",NaN,Germany,"Witten, North Rhine-Westphalia",Changed name,1967.0,"1966-1967 (as),1967-1986,1986-1993 (as)"
67636,Limelight,https://www.metal-archives.com/bands/Limelight...,NWOBHM/Heavy Metal/Hard Rock,"['Life', ' Feelings', ' Love', ' Thoughts']",NaN,United Kingdom,"Mansfield, Nottinghamshire, England",Split-up,1967.0,"1967-?,1997"
...,...,...,...,...,...,...,...,...,...,...
129176,כף הקלע,https://www.metal-archives.com/bands/%D7%9B%D7...,Black/Death Metal/Crust Punk,,Unsigned/independent,Israel,"Haifa, Haifa",Active,NaN,NaN
129197,ᛋᚢᚱᛏᚱ ᚹᛟᚢᚾᛞᛋ,https://www.metal-archives.com/bands/%E1%9B%8B...,"Raw Black Metal, Dark Ambient",,NaN,United Kingdom,NaN,Active,NaN,NaN
129208,こくまろみるく,https://www.metal-archives.com/bands/%E3%81%93...,Experimental Power Metal,"['The bizarre', ' Macabre']",Unsigned/independent,Japan,NaN,Active,NaN,NaN
129224,ノストラダムス,https://www.metal-archives.com/bands/%E3%83%8E...,Progressive Metal,,NaN,Japan,NaN,Split-up,NaN,NaN


# Vectorization

In [4]:
bands[bands['genre'].str.lower().str.contains('eastern')].head()

,name,url,genre,theme,label,country,location,status,date,years
87513,Pentagram,https://www.metal-archives.com/bands/Pentagram...,Thrash Metal (early); Heavy/Middle Eastern Fol...,"['Metal', ' War', ' Misery (early); Suicide', ...",NaN,Türkiye,Istanbul,Active,1987.0,"1987-present,2001-present (as)"
85566,Orphaned Land,https://www.metal-archives.com/bands/Orphaned_...,Doom/Death/Middle Eastern Folk Metal (early); ...,"['Abrahamic religions', ' Unity', ' Peace']",NaN,Israel,"Bat Yam, Tel Aviv / Petah Tikva, Central",Active,1992.0,"1991-1992 (as),1992-1997,2001-present"
72410,Melechesh,https://www.metal-archives.com/bands/Melechesh...,Black/Death/Middle Eastern Folk Metal,['Mesopotamian mythology'],NaN,International,Jerusalem/Bethlehem (early); Netherlands/Franc...,Active,1993.0,1993-present
14215,Blaakyum,https://www.metal-archives.com/bands/Blaakyum/...,Thrash/Heavy/Middle Eastern Folk Metal,"['Conceptual (thematic)', ' Symbolic', ' Socia...",Unsigned/independent,Lebanon,"Beirut / Antelias, Matn",Active,1995.0,"1995-2001,2007-present"
34862,Distorted,https://www.metal-archives.com/bands/Distorted...,Gothic/Melodic Death Metal with Middle Eastern...,"['Depression', ' Romance', ' Hypocrisy']",NaN,Israel,"Bat Yam, Tel Aviv",Active,1996.0,"1996-2010,2022-present"


In [5]:
bands[bands['theme'].str.lower().str.contains('early').fillna(False)].head()

,name,url,genre,theme,label,country,location,status,date,years
102499,Sir Lord Baltimore,https://www.metal-archives.com/bands/Sir_Lord_...,Heavy Metal/Hard Rock,"['Love', ' Women', ' Life (early); Christianit...",Unsigned/independent,United States,"New York, New York",Split-up,1968.0,"1968-1972,1977,2005-2006"
91937,Rainbow,https://www.metal-archives.com/bands/Rainbow/108,Hard Rock/Heavy Metal,"['Fantasy', ' Rock (early); Relationships', ' ...",NaN,United States,"Los Angeles, California / Connecticut / New York",Active,1975.0,"1975 (as),1976-1984,1994-1997,2015-present"
33522,Diamond Head,https://www.metal-archives.com/bands/Diamond_H...,"NWOBHM, Heavy Metal/Hard Rock","['Fantasy', ' Fiction', ' Love (early); Societ...",NaN,United Kingdom,"Stourbridge, West Midlands, England",Active,1976.0,"1976-1985,1991-1994,2000-present"
119193,V8,https://www.metal-archives.com/bands/V8/3236,Heavy Metal,"['Violence', ' Rebellion', ' Religion', ' Soci...",NaN,Argentina,Buenos Aires,Split-up,1979.0,"1979-1987,1996"
81146,Nightmare,https://www.metal-archives.com/bands/Nightmare...,Heavy/Power Metal,"['Life', ' Rock (early); Fantasy', ' Armageddo...",NaN,France,"Grenoble, Auvergne-Rhône-Alpes",Active,1979.0,"1979-1988,1999-present"


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vectorizer = CountVectorizer(binary=True, token_pattern="middle eastern|[0-9a-z'\-]+", min_df=10,
                             stop_words=['metal', 'rock', "'n'", 'roll', 'early', 'mid', 'later',
                                         'and', 'elements', 'influences', 'various', 'with'])
genre_mat = vectorizer.fit_transform(bands['genre'])
genre_names = vectorizer.get_feature_names_out()
print(genre_names)

['acoustic' 'alternative' 'ambient' 'aor' 'atmospheric' 'avant-garde'
 'black' 'blackened' 'blues' 'brutal' 'celtic' 'classical' 'crossover'
 'crust' 'crustcore' 'cybergrind' 'd-beat' 'dark' 'darkwave' 'death'
 'deathcore' 'depressive' 'djent' 'doom' 'drone' 'dungeon' 'electronic'
 'electronica' 'epic' 'experimental' 'extreme' 'folk' 'funeral' 'funk'
 'fusion' 'glam' 'goregrind' 'gothic' 'grind' 'grindcore' 'groove'
 'grunge' 'hard' 'hardcore' 'heavy' 'industrial' 'jazz' 'mathcore'
 'medieval' 'melodic' 'metalcore' 'middle eastern' 'neoclassical'
 'neofolk' 'noise' 'noisecore' 'noisegrind' 'nu-metal' 'nwobhm' 'oi'
 'operatic' 'pagan' 'pop' 'post-black' 'post-hardcore' 'post-metal'
 'post-punk' 'post-rock' 'power' 'powerviolence' 'progressive'
 'psychedelic' 'punk' 'rac' 'raw' 'shoegaze' 'shred' 'slam' 'sludge'
 'southern' 'speed' 'stoner' 'symphonic' 'synth' 'technical' 'thrash'
 'viking']


In [8]:
vectorizer = CountVectorizer(binary=True, token_pattern="[0-9a-z][0-9a-z'\-/\s]*[0-9a-z]", min_df=10,
                             stop_words=['early', 'mid', 'later'])
theme_mat = vectorizer.fit_transform(bands['theme'])
theme_names = vectorizer.get_feature_names_out()
print(theme_names.shape)

(1116,)


# 1. By Genre

In [9]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=30, random_state=0)
kmeans.fit(genre_mat)
kmeans.labels_

array([21, 21, 21, ..., 11,  5, 13], dtype=int32)

In [10]:
groups = bands.groupby(kmeans.labels_)

In [11]:
groups.size()

0     21605
1      6935
2     11471
3     11590
4       648
5      6109
6      3847
7      1273
8      6728
9      2056
10     3544
11     3296
12     1297
13     2132
14     1298
15    14089
16     1530
17     1259
18     2696
19     2404
20     4481
21     2555
22     2699
23     3552
24     2090
25     2639
26     1590
27     1647
28     1337
29     1074
dtype: int64

In [12]:
groups['name'].apply(lambda x: x.iloc[: 5].reset_index(drop=True)).unstack()

,0,1,2,3,4
0,Evil Blood,Skull Face,White Hell,Cerbero,Veneno Maldito
1,Dawn,Megiddo,Detestor,Seventh Angel,Sil Khannaz
2,Zoetrope,IG-Elit,Blind Illusion,Stone Vengeance,Exodus
3,Black Sabbath,Judas Priest,Night Sun,Back Out,La Banda de Osiris
4,Malicious Hate,Mortician,Rotten Minds,Antesser,Morbus
5,Rush,Yargos,Annex,Trytan,H. Kristal
6,Eryops,Bad Heaven,Exciter,Acid,Die Krupps
7,Celtic Frost,Salem,Bellzlleb,Goatlord,Valhall
8,Grinder Heavy Metal,Vulcano,Poison,Master,Mantas
9,Graveyard Rodeo,Nightmare Visions,In the Colonnades,Pik,Pozno


In [13]:
kmeans.cluster_centers_

array([[3.70284656e-04, 4.62855820e-05, 0.00000000e+00, ...,
        2.31427910e-04, 0.00000000e+00, 5.41541310e-03],
       [0.00000000e+00, 2.16294160e-03, 4.32588320e-04, ...,
        1.61499640e-02, 1.18240807e-01, 3.46070656e-03],
       [0.00000000e+00, 2.26658530e-03, 8.71763578e-05, ...,
        1.32508064e-02, 1.00000000e+00, 2.61529073e-04],
       ...,
       [0.00000000e+00, 5.46448087e-03, 0.00000000e+00, ...,
        1.39647845e-02, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.99177263e-03, 0.00000000e+00, ...,
        1.04712042e-02, 8.52655198e-02, 0.00000000e+00],
       [1.86219739e-03, 0.00000000e+00, 4.65549348e-03, ...,
        9.31098696e-04, 0.00000000e+00, 1.02420857e-02]])

In [14]:
top_pos = np.argsort(kmeans.cluster_centers_)[:, -1 : -6 : -1]
genre_names[top_pos]

array([['black', 'raw', 'atmospheric', 'depressive', 'symphonic'],
       ['melodic', 'death', 'black', 'thrash', 'progressive'],
       ['thrash', 'speed', 'power', 'progressive', 'metalcore'],
       ['heavy', 'melodic', 'progressive', 'speed', 'doom'],
       ['death', 'brutal', 'grindcore', 'technical', 'goregrind'],
       ['progressive', 'death', 'power', 'symphonic', 'melodic'],
       ['gothic', 'symphonic', 'post-black', 'goregrind', 'speed'],
       ['doom', 'black', 'death', 'atmospheric', 'funeral'],
       ['thrash', 'death', 'black', 'groove', 'progressive'],
       ['doom', 'death', 'gothic', 'atmospheric', 'funeral'],
       ['grindcore', 'death', 'black', 'crust', 'technical'],
       ['power', 'melodic', 'symphonic', 'speed', 'folk'],
       ['folk', 'black', 'pagan', 'viking', 'symphonic'],
       ['sludge', 'doom', 'post-metal', 'atmospheric', 'drone'],
       ['deathcore', 'death', 'technical', 'progressive', 'metalcore'],
       ['death', 'technical', 'groove', 'b

In [15]:
top = np.sort(kmeans.cluster_centers_)[:, -1 : -6 : -1]
np.round(top, 2)

array([[1.  , 0.05, 0.05, 0.04, 0.04],
       [1.  , 1.  , 0.12, 0.12, 0.05],
       [1.  , 0.07, 0.06, 0.04, 0.03],
       [1.  , 0.07, 0.05, 0.05, 0.03],
       [1.  , 1.  , 1.  , 0.02, 0.01],
       [1.  , 0.19, 0.14, 0.04, 0.03],
       [0.35, 0.24, 0.14, 0.08, 0.07],
       [1.  , 1.  , 0.2 , 0.08, 0.08],
       [1.  , 1.  , 0.08, 0.05, 0.03],
       [1.  , 1.  , 0.11, 0.06, 0.04],
       [1.  , 0.78, 0.04, 0.03, 0.02],
       [1.  , 0.16, 0.15, 0.1 , 0.03],
       [1.  , 0.38, 0.13, 0.09, 0.07],
       [1.  , 0.54, 0.12, 0.07, 0.05],
       [1.  , 0.32, 0.13, 0.07, 0.07],
       [1.  , 0.05, 0.04, 0.03, 0.02],
       [1.  , 1.  , 0.06, 0.04, 0.02],
       [1.  , 0.96, 0.19, 0.13, 0.09],
       [0.65, 0.53, 0.05, 0.05, 0.03],
       [1.  , 1.  , 0.09, 0.05, 0.05],
       [1.  , 1.  , 0.04, 0.03, 0.01],
       [1.  , 0.99, 0.12, 0.03, 0.03],
       [1.  , 0.56, 0.23, 0.06, 0.03],
       [1.  , 1.  , 0.1 , 0.07, 0.07],
       [1.  , 1.  , 0.05, 0.03, 0.03],
       [1.  , 0.24, 0.14,

In [16]:
from scipy.sparse import csr_matrix

selector = csr_matrix((np.ones(genre_mat.shape[0]), (kmeans.labels_, np.arange(genre_mat.shape[0]))),
                      (kmeans.cluster_centers_.shape[0], genre_mat.shape[0]))
theme_by_genre = selector / selector.sum(axis=1) * theme_mat

In [17]:
top_pos = np.argsort(theme_by_genre)[:, -1 : -6 : -1]
theme_names[top_pos]

array([['death', 'satanism', 'darkness', 'war', 'nature'],
       ['death', 'life', 'war', 'darkness', 'society'],
       ['war', 'death', 'violence', 'society', 'politics'],
       ['life', 'fantasy', 'war', 'metal', 'death'],
       ['gore', 'death', 'violence', 'sickness', 'porn'],
       ['instrumental', 'life', 'fantasy', 'death', 'philosophy'],
       ['death', 'love', 'gore', 'life', 'nature'],
       ['death', 'darkness', 'depression', 'nature', 'occultism'],
       ['death', 'war', 'violence', 'society', 'politics'],
       ['death', 'depression', 'sorrow', 'darkness', 'despair'],
       ['gore', 'death', 'violence', 'politics', 'humour'],
       ['fantasy', 'life', 'love', 'war', 'christianity'],
       ['nature', 'paganism', 'folklore', 'fantasy', 'history'],
       ['death', 'instrumental', 'depression', 'drugs', 'life'],
       ['death', 'violence', 'gore', 'murder', 'anti-religion'],
       ['death', 'gore', 'war', 'violence', 'horror'],
       ['death', 'war', 'satanism'

In [18]:
top = np.sort(theme_by_genre)[:, -1 : -6 : -1]
np.round(top, 2)

array([[0.13, 0.13, 0.12, 0.08, 0.07],
       [0.12, 0.04, 0.04, 0.04, 0.03],
       [0.11, 0.1 , 0.07, 0.06, 0.05],
       [0.06, 0.05, 0.04, 0.04, 0.03],
       [0.45, 0.2 , 0.08, 0.07, 0.06],
       [0.09, 0.06, 0.04, 0.04, 0.03],
       [0.07, 0.06, 0.05, 0.04, 0.04],
       [0.18, 0.09, 0.09, 0.07, 0.06],
       [0.17, 0.09, 0.06, 0.04, 0.03],
       [0.2 , 0.08, 0.07, 0.07, 0.05],
       [0.18, 0.14, 0.06, 0.06, 0.05],
       [0.19, 0.07, 0.04, 0.04, 0.03],
       [0.18, 0.13, 0.08, 0.08, 0.07],
       [0.06, 0.04, 0.03, 0.03, 0.03],
       [0.11, 0.05, 0.05, 0.04, 0.03],
       [0.23, 0.1 , 0.05, 0.05, 0.04],
       [0.19, 0.15, 0.12, 0.09, 0.09],
       [0.18, 0.14, 0.11, 0.1 , 0.07],
       [0.04, 0.04, 0.03, 0.03, 0.02],
       [0.11, 0.07, 0.06, 0.04, 0.04],
       [0.19, 0.1 , 0.1 , 0.09, 0.07],
       [0.09, 0.08, 0.03, 0.03, 0.03],
       [0.06, 0.05, 0.03, 0.03, 0.02],
       [0.35, 0.22, 0.08, 0.06, 0.06],
       [0.05, 0.05, 0.04, 0.04, 0.03],
       [0.12, 0.06, 0.06,

# 2. By Theme

In [19]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=30, random_state=0)
kmeans.fit(theme_mat)
kmeans.labels_

array([ 0, 16, 27, ...,  0,  0,  0], dtype=int32)

In [20]:
groups = bands.groupby(kmeans.labels_)

In [21]:
groups.size()

0     78896
1       338
2       689
3      2156
4      1659
5      2171
6      1656
7      1053
8      1046
9      1035
10     1898
11     1661
12     2248
13     3518
14     6828
15     1831
16      691
17     1196
18     2323
19      989
20     2107
21      143
22      643
23     2014
24     1118
25     2688
26     1750
27     2386
28     2199
29      541
dtype: int64

In [22]:
groups['name'].apply(lambda x: x.iloc[: 5].reset_index(drop=True)).unstack()

,0,1,2,3,4
0,Olympic,Cool Feet,Ash,Flower Travellin' Band,Night Sun
1,Annihilator,Systematic Marsmurder,Pungent Stench,Antropomorphia,Suppurated Fetus
2,Sodom,Executioner,Headcrasher,Krânio Metálico,Mayhem
3,Kat,Semen of Satan,Antichrist,Bathory,RetroSatan
4,Mantas,Necrophagia,Death,Genocide,Xecutioner
5,Life Discussion,Der Rache liebt,Mortified,Cripple Bastards,Eyehategod
6,LTP,Emerald,Stairway,Sword,Trytan
7,Graf Spee,Slayer,Sagrado Inferno,Manimals,Burn
8,Blast,Murasaki,Pentagram,Twisted Sister,Ritual
9,Trigger Zone,Oral,Maple Cross,Necromancia,Hämmer


In [23]:
top_pos = np.argsort(kmeans.cluster_centers_)[:, -1 : -6 : -1]
theme_names[top_pos]

array([['depression', 'philosophy', 'hate', 'paganism', 'social issues'],
       ['perversion', 'gore', 'sex', 'death', 'blasphemy'],
       ['war', 'satanism', 'hate', 'death', 'anti-christianity'],
       ['satanism', 'death', 'darkness', 'blasphemy', 'evil'],
       ['death', 'gore', 'violence', 'horror', 'torture'],
       ['misanthropy', 'death', 'darkness', 'hate', 'depression'],
       ['christianity', 'life', 'death', 'god', 'faith'],
       ['evil', 'darkness', 'death', 'satanism', 'blasphemy'],
       ['love', 'death', 'sadness', 'sorrow', 'depression'],
       ['hatred', 'death', 'war', 'violence', 'national socialism'],
       ['society', 'death', 'war', 'violence', 'religion'],
       ['instrumental', 'death', 'depression', 'melancholy', 'space'],
       ['violence', 'death', 'war', 'hate', 'metal'],
       ['war', 'death', 'destruction', 'religion', 'metal'],
       ['death', 'darkness', 'life', 'hate', 'horror'],
       ['anti-christianity', 'satanism', 'blasphemy', 'dar

In [24]:
top = np.sort(kmeans.cluster_centers_)[:, -1 : -6 : -1]
np.round(top, 2)

array([[0.01, 0.01, 0.01, 0.01, 0.01],
       [1.  , 0.62, 0.21, 0.21, 0.09],
       [1.  , 0.68, 0.4 , 0.17, 0.16],
       [1.  , 0.18, 0.17, 0.11, 0.05],
       [1.  , 1.  , 0.11, 0.1 , 0.08],
       [1.  , 0.19, 0.11, 0.11, 0.11],
       [1.  , 0.02, 0.02, 0.02, 0.02],
       [1.  , 0.65, 0.17, 0.08, 0.07],
       [1.  , 0.25, 0.1 , 0.08, 0.06],
       [1.  , 0.35, 0.22, 0.11, 0.07],
       [1.  , 0.15, 0.11, 0.08, 0.07],
       [1.  , 0.  , 0.  , 0.  , 0.  ],
       [1.  , 0.39, 0.19, 0.1 , 0.05],
       [1.  , 0.39, 0.06, 0.04, 0.04],
       [1.  , 0.13, 0.08, 0.05, 0.05],
       [1.  , 0.34, 0.16, 0.1 , 0.08],
       [0.68, 0.5 , 0.19, 0.12, 0.09],
       [1.  , 0.19, 0.07, 0.05, 0.05],
       [1.  , 0.13, 0.12, 0.07, 0.05],
       [1.  , 0.29, 0.18, 0.08, 0.07],
       [1.  , 0.24, 0.19, 0.18, 0.05],
       [1.  , 0.45, 0.29, 0.14, 0.07],
       [1.  , 0.45, 0.16, 0.13, 0.08],
       [1.  , 0.13, 0.07, 0.06, 0.05],
       [1.  , 0.33, 0.17, 0.12, 0.12],
       [1.  , 0.11, 0.06,

In [25]:
from scipy.sparse import csr_matrix

selector = csr_matrix((np.ones(theme_mat.shape[0]), (kmeans.labels_, np.arange(theme_mat.shape[0]))),
                      (kmeans.cluster_centers_.shape[0], theme_mat.shape[0]))
genre_by_theme = selector / selector.sum(axis=1) * genre_mat

In [26]:
top_pos = np.argsort(genre_by_theme)[:, -1 : -6 : -1]
genre_names[top_pos]

array([['death', 'thrash', 'black', 'heavy', 'melodic'],
       ['death', 'brutal', 'grindcore', 'black', 'goregrind'],
       ['black', 'death', 'thrash', 'raw', 'melodic'],
       ['black', 'death', 'thrash', 'raw', 'doom'],
       ['death', 'brutal', 'grindcore', 'thrash', 'goregrind'],
       ['black', 'death', 'doom', 'thrash', 'melodic'],
       ['death', 'heavy', 'black', 'thrash', 'power'],
       ['black', 'death', 'thrash', 'heavy', 'melodic'],
       ['heavy', 'gothic', 'doom', 'death', 'melodic'],
       ['black', 'death', 'thrash', 'melodic', 'doom'],
       ['thrash', 'death', 'heavy', 'melodic', 'groove'],
       ['progressive', 'doom', 'black', 'death', 'stoner'],
       ['thrash', 'death', 'black', 'heavy', 'grindcore'],
       ['death', 'thrash', 'black', 'heavy', 'melodic'],
       ['death', 'black', 'thrash', 'doom', 'melodic'],
       ['black', 'death', 'thrash', 'melodic', 'raw'],
       ['thrash', 'heavy', 'death', 'black', 'speed'],
       ['death', 'thrash', 'b

In [27]:
top = np.sort(genre_by_theme)[:, -1 : -6 : -1]
np.round(top, 2)

array([[0.33, 0.22, 0.19, 0.18, 0.11],
       [0.81, 0.43, 0.23, 0.17, 0.14],
       [0.82, 0.24, 0.15, 0.06, 0.03],
       [0.87, 0.21, 0.09, 0.04, 0.03],
       [0.93, 0.28, 0.19, 0.08, 0.07],
       [0.75, 0.23, 0.07, 0.06, 0.04],
       [0.29, 0.26, 0.17, 0.16, 0.14],
       [0.64, 0.3 , 0.16, 0.09, 0.06],
       [0.32, 0.27, 0.18, 0.16, 0.15],
       [0.54, 0.37, 0.18, 0.05, 0.04],
       [0.4 , 0.38, 0.16, 0.1 , 0.08],
       [0.36, 0.19, 0.15, 0.12, 0.1 ],
       [0.5 , 0.47, 0.11, 0.07, 0.06],
       [0.38, 0.35, 0.32, 0.1 , 0.07],
       [0.55, 0.31, 0.19, 0.12, 0.1 ],
       [0.78, 0.31, 0.1 , 0.04, 0.03],
       [0.32, 0.27, 0.22, 0.19, 0.1 ],
       [0.52, 0.21, 0.16, 0.13, 0.11],
       [0.69, 0.15, 0.14, 0.12, 0.11],
       [0.32, 0.25, 0.24, 0.15, 0.14],
       [0.74, 0.25, 0.12, 0.07, 0.04],
       [0.53, 0.48, 0.16, 0.07, 0.06],
       [0.49, 0.49, 0.2 , 0.07, 0.04],
       [0.66, 0.26, 0.11, 0.11, 0.07],
       [0.67, 0.28, 0.25, 0.08, 0.06],
       [0.37, 0.31, 0.14,

# 3. By Year

In [28]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=30, n_init=50, random_state=0)
kmeans.fit(pd.DataFrame(bands['date']).dropna())
kmeans.labels_

array([20, 20, 20, ..., 15, 15, 15], dtype=int32)

In [29]:
groups = bands[~bands['date'].isna()].groupby(kmeans.labels_)

In [30]:
groups.size()

0     5700
1     4507
2     6913
3     3537
4     4730
5     2707
6     5520
7     5689
8     1102
9     5913
10    3449
11    4141
12    4015
13    5159
14    5092
15    4100
16    4167
17    5209
18    5851
19    5969
20     128
21    5348
22    4984
23    1091
24    2656
25    4443
26    3816
27    5032
28    3519
29    3828
dtype: int64

In [31]:
groups['name'].apply(list)

0     [Oxidized Faith, Blood Harvest, Assaulter, Nec...
1     [Brazen, Untory, Thine, Clandestine, Man to Ma...
2     [Doedskvad, Dying Giants, Necrovaginal Prolaps...
3     [Indwelling, EoB, Apostasy, Estuary, Zan, Esti...
4     [Лесьяр, Semynar, Sept of Memnon, Undead Night...
5     [Sudden Impact, Fatal Violence, Diatribe, Glös...
6     [Ethereal Darkness, Akiavel, Militant, OP, See...
7     [Unhale, Imperium, Dark O Malex, Ultor, Dark P...
8     [Quartz, Vardis, Sacred Few, Discharge, Rock G...
9     [Dargaard, Thorium, Devenir, Total Hate, Desce...
10    [Absurter Cryst, Sceptic Age, Abortion, Condol...
11    [High n' Heavy, Stink Tank, Thousand Arrows, H...
12    [Magus, Wanted, Bonesawer, Defy Christ, Hospic...
13    [Depurity, Derkaizer, Angel's Storm, From Hell...
14    [Nothing, Sanecra, Ampedra, Ordalium, Voodoo K...
15    [Fenris Vrede, Slaveknight, Fentanil, Moral Co...
16    [Hordes of Yore, Satyrica, Disrepute, Left to ...
17    [Collateral Bleeding, Abnormity, Zombie Ri

In [32]:
kmeans.cluster_centers_

array([[2007.        ],
       [1993.50832039],
       [2016.47982063],
       [2000.        ],
       [2011.        ],
       [1985.11932028],
       [2018.4701087 ],
       [2004.        ],
       [1979.78130672],
       [1997.53543041],
       [1989.50652363],
       [2014.        ],
       [1991.53225405],
       [2009.        ],
       [1995.51060487],
       [2020.48      ],
       [2002.        ],
       [2003.        ],
       [2006.        ],
       [2005.        ],
       [1973.3828125 ],
       [2008.        ],
       [2012.        ],
       [1982.56553621],
       [1987.51656627],
       [2013.        ],
       [2001.        ],
       [2010.        ],
       [1999.        ],
       [2015.        ]])

In [33]:
bands[bands['name'] == 'Hatred Sculpted Souls']

,name,url,genre,theme,label,country,location,status,date,years
52871,Hatred Sculpted Souls,https://www.metal-archives.com/bands/Hatred_Sc...,Death Metal,,Unsigned/independent,Brazil,"Carlos Barbosa, Rio Grande do Sul",Active,2013.0,2013-present
